In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from tqdm import tqdm
import math

### Обработка входных данных

In [2]:
df = pd.read_csv('1coinUSD.csv', header=None)
df.head()

,0,1,2
0,1394351059,621.0,0.01
1,1394351073,620.0,0.01
2,1394427477,620.0,0.01
3,1394427596,620.5,0.01
4,1394427614,621.0,0.01


In [3]:
def normalize(arr):
    max_val = np.max(arr)
    min_val = np.min(arr)
    arr = (arr - min_val) / max_val
    return arr

coin_val = normalize(df.values[:,1])[:100000]
print(coin_val.shape, type(coin_val), coin_val.dtype)

(100000,) <class 'numpy.ndarray'> float64


### Класс подсчета параметров для построения ВПФР нового ряда

In [62]:
class model_param:
    def __init__(self, var, T=10000, count_velocity = True, n_v=100):
        
        self.var = var
        self.delta = self.count_delta()

        self.T = T
        self.n = math.floor(math.pow(T, 0.34))
        self.n_v = n_v

        self.density = []
        self.velocity = []

        self.current_density = None
        self.current_velocity_table = None
        self.current_velocity = None

        self.gist_density = None
        self.gist_velocity = None
        
        self.current_density_bin_to_points = [[] for _ in range(self.n)]
        self.current_density_points_to_bin = np.array([-1 for _ in range(var.shape[0])])
        self.current_velocity_points_to_bin = np.array([-1 for _ in range(var.shape[0])])

        self.mean_velocity_in_bin = None
        
        self.count_velocity = count_velocity
    
    def count_delta(self):
        return np.append(self.var[1:] - self.var[:-1], 0)
    
    def run(self):
        for i in tqdm(range(self.T - 1, self.var.shape[0])):
            self.get_current_density(i)
            self.density.append(sparse.csr_matrix(self.current_density))
            if (self.count_velocity):
                self.get_current_velocity(i)
                self.velocity.append(sparse.csr_matrix(self.current_velocity))
        self.density = sparse.vstack(self.density, format='csr')
        if (self.count_velocity):
            self.velocity = sparse.vstack(self.velocity, format='csr')

        
    def get_current_velocity(self, i):
        self.current_velocity = np.zeros(self.n)
        if (self.current_velocity_table is None):
            self.count_init_velocity_table()
        else:
            self.get_next_velocity_table(i)
        for i in range(self.n):
            if (self.current_density[i] == 0):
                self.current_velocity[i] = 0
            else:
                integral = np.multiply(self.current_velocity_table[:, i], self.mean_velocity_in_bin)
                self.current_velocity[i] = (integral).sum() / self.current_density[i]
            
    def get_next_velocity_table(self, i):
        self.current_velocity_points_to_bin[i] = self.get_bin(self.gist_velocity, self.delta[i])      
        self.current_velocity_table[self.current_velocity_points_to_bin[i - self.T]][self.current_density_points_to_bin[i - self.T]] -= 1
        self.current_velocity_table[self.current_velocity_points_to_bin[i]][self.current_density_points_to_bin[i]] += 1
            
    def count_init_velocity_table(self):
        var = self.delta[:self.T]
        self.current_velocity_table = np.zeros((self.n_v, self.n))
        if (self.gist_velocity is None):
            self.gist_velocity = self.get_gist(-1, 1, self.n_v)
            self.mean_velocity_in_bin = (self.gist_velocity[:, 1] + self.gist_velocity[:, 0]) / 2
        for i, g in enumerate(self.gist_velocity):
            for j in range(self.n):
                current_var = var[self.current_density_bin_to_points[j]]
                indicies = np.nonzero(np.logical_and(current_var >= g[0], current_var < g[1]))[0]
                if (len(indicies)):
                    self.current_velocity_points_to_bin[np.array(self.current_density_bin_to_points[j])[indicies]] = i
                self.current_velocity_table[i][j] = len(indicies)
    
    def get_current_density(self, i):
        if (self.current_density is None):
            self.count_init_density()
        else:
            self.get_next_density(i)
            
    def get_next_density(self, i):
        self.current_density[self.current_density_points_to_bin[i - self.T]] -= 1 / self.T
        self.current_density_points_to_bin[i] = self.get_bin(self.gist_density, self.var[i])
        self.current_density[self.current_density_points_to_bin[i]] += 1 / self.T
        
    def get_bin(self, gist, elem):
        for i, g in enumerate(gist):
            if (g[0] <= elem < g[1]):
                return i
        
    def count_init_density(self):
        var = self.var[:self.T]
        self.current_density = np.zeros(self.n)
        if (self.gist_density is None):
            self.gist_density = self.get_gist(0, 1, self.n)
        for i, g in enumerate(self.gist_density):
            indicies = np.nonzero(np.logical_and(var >= g[0], var < g[1]))[0]
            self.current_density_bin_to_points[i].extend(indicies)
            self.current_density_points_to_bin[indicies] = i
            self.current_density[i] = len(indicies) / self.T
        self.current_density_bin_to_points = np.array(self.current_density_bin_to_points)
        
            
    def get_gist(self, a, b, n):
        return np.array([(j * (b - a) / n + a, (j + 1) * (b - a) / n + a) for j in range(n)], dtype=float)

Тестирование функций класса 

In [43]:
a = model_param(np.array([1, 13, -8, 3, 6]))
print(a.delta)

[ 12 -21  11   3   0]


In [100]:
b = model_param(np.array([1]), T = 8)
print(b.n)
print(b.delta)
b.run()
print(b.gist_density)

3
[0]
[[0.         0.33333333]
 [0.33333333 0.66666667]
 [0.66666667 1.        ]]


In [58]:
c = model_param(np.arange(0, 10) / 10, T=9)
print(c.var)
print(c.n)
print(c.delta)
c.run()
print(c.gist_density)
print(c.current_density_bin_to_points)
print(c.current_density_points_to_bin)
print(c.current_density)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
4
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0. ]
[[0.   0.25]
 [0.25 0.5 ]
 [0.5  0.75]
 [0.75 1.  ]]
[list([0, 1, 2]) list([3, 4]) list([5, 6, 7]) list([8])]
[0 0 0 1 1 2 2 2 3 3]
[2. 2. 3. 2.]


In [59]:
d = model_param(np.arange(0, 100) / 100, T=10)
d.run()
#print(d.current_density_points_to_bin)
print(d.density.toarray())

[[10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [ 9.  1.  0.  0.]
 [ 8.  2.  0.  0.]
 [ 7.  3.  0.  0.]
 [ 6.  4.  0.  0.]
 [ 5.  5.  0.  0.]
 [ 4.  6.  0.  0.]
 [ 3.  7.  0.  0.]
 [ 2.  8.  0.  0.]
 [ 1.  9.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0.  9.  1.  0.]
 [ 0.  8.  2.  0.]
 [ 0.  7.  3.  0.]
 [ 0.  6.  4.  0.]
 [ 0.  5.  5.  0.]
 [ 0.  4.  6.  0.]
 [ 0.  3.  7.  0.]
 [ 0.  2.  8.  0.]
 [ 0.  1.  9.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0. 10

In [155]:
d = model_param(np.arange(0, 10) / 10, T=10, n_v=10)
d.run()
print(d.gist_velocity)

[[-1.  -0.8]
 [-0.8 -0.6]
 [-0.6 -0.4]
 [-0.4 -0.2]
 [-0.2  0. ]
 [ 0.   0.2]
 [ 0.2  0.4]
 [ 0.4  0.6]
 [ 0.6  0.8]
 [ 0.8  1. ]]


In [44]:
e = model_param(np.arange(0, 10) / 10, T=10, n_v=2)
e.run()
print(e.current_velocity_table)

[[0. 0. 0. 0.]
 [3. 2. 3. 2.]]


In [55]:
f = model_param(np.arange(0, 100) / 100, T=10, n_v=4)
f.run()
print(f.density.toarray())
print(f.velocity.toarray())

[[10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [10.  0.  0.  0.]
 [ 9.  1.  0.  0.]
 [ 8.  2.  0.  0.]
 [ 7.  3.  0.  0.]
 [ 6.  4.  0.  0.]
 [ 5.  5.  0.  0.]
 [ 4.  6.  0.  0.]
 [ 3.  7.  0.  0.]
 [ 2.  8.  0.  0.]
 [ 1.  9.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0.  9.  1.  0.]
 [ 0.  8.  2.  0.]
 [ 0.  7.  3.  0.]
 [ 0.  6.  4.  0.]
 [ 0.  5.  5.  0.]
 [ 0.  4.  6.  0.]
 [ 0.  3.  7.  0.]
 [ 0.  2.  8.  0.]
 [ 0.  1.  9.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0. 10

### Рассчет параметров модели

In [63]:
params = model_param(coin_val, T=5000)
params.run()

100%|██████████| 95001/95001 [01:32<00:00, 1023.02it/s]


In [64]:
sparse.save_npz("velocity_matrix_csr.npz", params.velocity)
sparse.save_npz("density_matrix_csr.npz", params.density)

In [65]:
print(params.density[45].toarray())

[[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.2546 0.306  0.4394 0.     0.     0.     0.     0.    ]]


### Построение модели эволюции системы

In [75]:
class evolutionary:
    def __init__(self, var, n, T, density, velocity, density_gist):
        self.var = var
        self.n = n
        self.T = T
        self.density = density
        self.velocity = velocity
        self.density_gist = density_gist
        self.new_density = []
        self.init_density = None
        
    def run(self):
        self.init_density = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5200622, 0.4259711, 0.05396671, 0., 0., 0., 0., 0.])
        self.new_density.append(sparse.csr_matrix(self.init_density))
        self.new_density.append(sparse.csr_matrix(self.init_density))
        for i in tqdm(range(1, self.var.shape[0] -self.T - 1)):
            current_density = self.density[i].toarray()[0]
            new_velocity = self.get_new_velocity(i)
            shifted_density = self.get_shifted(current_density)
            shifted_new_velocity = self.get_shifted(new_velocity)
            new_current_density = current_density \
                                  + np.multiply(current_density, new_velocity) \
                                  - np.multiply(shifted_density, shifted_new_velocity)
            new_current_density[new_current_density < 0] = 0
            assert (new_current_density < 0).sum() == 0
            self.new_density.append(sparse.csr_matrix(new_current_density / new_current_density.sum()))
        self.new_density = sparse.vstack(self.new_density, format='csr')        
                    
    def get_new_velocity(self, i):
        current_density = self.density[i].toarray()[0]
        prev_density = self.density[i - 1].toarray()[0]
        prev_velocity = self.velocity[i - 1].toarray()[0]
        new_velocity = np.zeros(self.n)
        indicies = np.nonzero(current_density)
        new_velocity[indicies] = np.divide(np.multiply(prev_density[indicies], prev_velocity[indicies]), current_density[indicies])
        return new_velocity
    
    def get_shifted(self, array):
        return np.append(array[1:], 0)

### Расчет эволюционирующих ВПФР

In [76]:
density_evolution = evolutionary(coin_val, params.n, params.T, params.density, params.velocity, params.gist_density)
density_evolution.run()
sparse.save_npz("new_density_matrix_csr.npz", density_evolution.new_density)

100%|██████████| 94998/94998 [01:48<00:00, 878.52it/s]


In [78]:
print(density_evolution.new_density.shape)
print(params.velocity.shape)
print(params.density.shape)

(95000, 18)
(95001, 18)
(95001, 18)


In [81]:
print(density_evolution.new_density[1].toarray())
print()
print(density_evolution.new_density[100].toarray())
print()
print(density_evolution.new_density[1800].toarray())

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.5200622  0.4259711
  0.05396671 0.         0.         0.         0.         0.        ]]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.48894325 0.34822295
  0.1628338  0.         0.         0.         0.         0.        ]]


### Генерация ряда на основе ВПФР

In [131]:
class generated_trajectory:
    def __init__(self, density, T, n, N, generation):
        self.f = density
        self.T = T
        self.N = N
        self.n = n
        self.j = generation
        self.y = np.random.uniform(0, 1, self.N)
        self.x = np.zeros(N)
        self.count = 0
        
    def count_x(self):
        for i in tqdm(range(self.N)):
            j = self.f[i].toarray()[0].nonzero()[0]
            if (self.f[i].toarray()[0][self.j] != 0):
                assert self.n is not None
                assert (self.f[i].toarray()[0][:self.j + 1]).sum() is not None
                assert self.y is not None
                assert self.y[i] is not None
                self.x[i] = 1 / self.n * ((self.y[i] - (self.f[i].toarray()[0][:self.j]).sum()) / self.f[i][self.j] + self.j - 1)
            else:
                self.count += 1

In [132]:
new_traj = generated_trajectory(density_evolution.new_density, density_evolution.T, density_evolution.n, 90000, 12)
new_traj.count_x()




  0%|          | 0/90000 [00:00<?, ?it/s]




IndexError: index (12) out of range

In [120]:
new_traj.count

63997


 71%|███████   | 63749/90000 [00:29<00:11, 2191.46it/s]